In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !wget https://zenodo.org/record/7007358/files/glow19_pnasnexus.csv

In [ ]:
from pathlib import Path
import logging
import os

import numpy as np
import pandas as pd
from hydra import compose, initialize
# from omegaconf import OmegaConf
# import matplotlib.pyplot as plt

# from physped.io.readers import trajectory_reader
from physped.omegaconf_resolvers import register_new_resolvers
from physped.preprocessing.trajectories import preprocess_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.visualization.plot_trajectories import plot_trajectories
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
# from physped.io.readers import read_grid_bins
from physped.io.writers import save_piecewise_potential
from physped.visualization.plot_histograms import create_all_histograms, plot_multiple_histograms

log = logging.getLogger(__name__)

In [ ]:
with initialize(version_base=None, config_path="conf", job_name = "test_app"):
    cfg = compose(
        config_name="config"
        , return_hydra_config=True
        , overrides=["params=custom_paths"]
        )
    print(cfg)
register_new_resolvers()

In [ ]:
trajectories = pd.read_csv('glow19_pnasnexus.csv')
trajectories['time'] = pd.to_datetime(trajectories['time'])
# trajectories = trajectories[['time', 'tracked_object', 'x', 'y']].copy()
trajectories.head(3)

In [ ]:
os.chdir(cfg.root_dir / 'outputs' / 'custom_paths')

In [ ]:
# Update parameters
# cfg.params.dt = 0.0333333333333
cfg.params.fps = 30
cfg.params.colnames.xf = 'x_sav'
cfg.params.colnames.yf = 'y_sav'
cfg.params.colnames.Pid = 'tracked_object'
cfg.params.colnames.time = 'time'

# Infer the edge of the measurement domain from the data
xmin = int(np.floor(trajectories.x.min())) - 1
xmax = int(np.ceil(trajectories.x.max())) + 1
ymin = int(np.floor(trajectories.y.min())) - 1
ymax = int(np.ceil(trajectories.y.max())) + 1
cfg.params.trajectory_plot.xlims = [xmin, xmax]
cfg.params.trajectory_plot.ylims = [ymin, ymax]

In [ ]:
# Preprocess trajectories
preprocessed_trajectories = preprocess_trajectories(trajectories, config=cfg)
preprocessed_trajectories.head(3)

In [ ]:
cfg['params']

In [ ]:
plot_trajectories(preprocessed_trajectories, cfg, 'recorded')

In [ ]:
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, cfg)
save_piecewise_potential(piecewise_potential, Path.cwd())

In [ ]:
# Simulate new trajectories using the learned potential
simulated_trajectories = simulate_trajectories(piecewise_potential, cfg)
simulated_trajectories.head(3)

In [ ]:
plot_trajectories(simulated_trajectories, cfg, 'simulated')  

In [ ]:

cfg.params.histogram_plot.xlims = [xmin, xmax]
cfg.params.histogram_plot.ylims = [ymin, ymax]
cfg.params.histogram_plot.ylims = [ymin, ymax]
# Create histograms
observables = ["xf", "yf", "uf", "vf"]
histograms = create_all_histograms(preprocessed_trajectories, simulated_trajectories, observables = observables)

# Plot Histograms
plot_multiple_histograms(observables, histograms, "PDF", cfg)